In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ============================================================
# SENTETIKLE FINE-TUNE EDILMIS MODEL -> SADECE REAL TEST
# ============================================================

import os, json, random, numpy as np, cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T, models
from tqdm import tqdm

# ---------------- Paths ----------------
REAL_ROOT = "/content/drive/MyDrive/dataset_split/real_gaze_vectors_videos_and_texts_split"

# BURAYI: sentetik fine-tune sonucunda kaydettiğin checkpoint'e göre değiştir
CKPT_PATH = "/content/drive/MyDrive/dataset_split/best_gaze_synthetic_from_real_lastpt_resnet50.pt"

# ---------------- Env ----------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)
torch.manual_seed(42)
if device == "cuda":
    torch.cuda.manual_seed_all(42)
np.random.seed(42)
random.seed(42)

# ============================================================
# Yardımcı: Angular Error
# ============================================================

def angular_error_deg(pred, gt):
    pred = F.normalize(pred, dim=1)
    gt   = F.normalize(gt,   dim=1)
    cos_sim = torch.clamp(torch.sum(pred * gt, dim=1), -1.0, 1.0)
    return torch.rad2deg(torch.acos(cos_sim))

# ============================================================
# REAL TEyeD Dataset
# ============================================================

class TEyeDRealDataset(Dataset):
    """
    REAL TEyeD:
      REAL_ROOT/test/*.mp4
      REAL_ROOT/test/*.mp4gaze_vec.txt

    TXT satır formatı:
      FRAME;X;Y;Z;
      1;-0.0218;-0.0039;0.9997;
      ...
    """
    def __init__(self, root_dir, split="test", transform=None):
        self.root_dir = os.path.join(root_dir, split)
        self.transform = transform
        self.samples = []  # (video_path, frame_idx, (gx,gy,gz))

        if not os.path.exists(self.root_dir):
            print(f"⚠️ REAL path not found: {self.root_dir}")
            return

        video_files = sorted([f for f in os.listdir(self.root_dir) if f.lower().endswith(".mp4")])
        print(f"[REAL-{split}] found {len(video_files)} videos.")

        for vname in tqdm(video_files, desc=f"Indexing REAL-{split}"):
            vpath = os.path.join(self.root_dir, vname)
            txt_path = vpath + "gaze_vec.txt"
            if not os.path.exists(txt_path):
                print(f"⚠️ Gaze file missing for {vname}")
                continue

            frame_ids, gazes = self._load_gaze_file(txt_path)

            cap = cv2.VideoCapture(vpath)
            if not cap.isOpened():
                print(f"⚠️ Could not open video: {vpath}")
                continue
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            cap.release()

            L = min(total_frames, len(frame_ids))
            for i in range(L):
                self.samples.append((vpath, int(frame_ids[i]), gazes[i]))

        print(f"REAL-{split}: {len(self.samples)} samples")

    def _load_gaze_file(self, txt_path):
        frame_ids = []
        gazes = []
        with open(txt_path, "r") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                if line.upper().startswith("FRAME"):
                    continue
                parts = line.split(";")
                if len(parts) < 4:
                    continue
                try:
                    fid = int(float(parts[0]))
                    gx = float(parts[1]); gy = float(parts[2]); gz = float(parts[3])
                except ValueError:
                    continue
                frame_ids.append(max(0, fid - 1))  # 1-based -> 0-based
                gazes.append([gx, gy, gz])
        frame_ids = np.array(frame_ids, dtype=np.int64)
        gazes = np.array(gazes, dtype=np.float32)
        return frame_ids, gazes

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        vpath, fidx, gaze = self.samples[idx]
        cap = cv2.VideoCapture(vpath)
        if not cap.isOpened():
            raise RuntimeError(f"Could not open video {vpath}")
        cap.set(cv2.CAP_PROP_POS_FRAMES, fidx)
        ok, frame = cap.read()
        cap.release()
        if not ok:
            raise RuntimeError(f"Could not read frame {fidx} from {vpath}")
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            frame = self.transform(frame)

        tgt = torch.tensor(gaze, dtype=torch.float32)
        return frame, tgt

# ============================================================
# Transform, DataLoader
# ============================================================

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

real_tf_eval = T.Compose([
    T.ToPILImage(),
    T.Resize((224, 224)),
    # İstersen burada T.Grayscale(num_output_channels=3) ekleyebilirsin
    T.ToTensor(),
    T.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

BATCH_SIZE = 8
NUM_WORKERS = 4 if device == "cuda" else 2

def make_loader(ds, shuffle=False):
    return DataLoader(
        ds,
        batch_size=BATCH_SIZE,
        shuffle=shuffle,
        num_workers=NUM_WORKERS,
        pin_memory=(device=="cuda"),
        persistent_workers=(device=="cuda" and NUM_WORKERS>0 and len(ds)>0),
        prefetch_factor=4 if device=="cuda" else 2,
    )

real_test_dataset = TEyeDRealDataset(REAL_ROOT, "test", real_tf_eval)
real_test_loader  = make_loader(real_test_dataset, shuffle=False)

print(f"REAL Test samples: {len(real_test_dataset)}")

# ============================================================
# Model: ResNet50 -> 3D gaze, sentetik-finetune checkpoint'ini yükle
# ============================================================

gaze_model = models.resnet50(weights=None)  # Ağırlıklar checkpoint'ten gelecek
gaze_model.fc = nn.Linear(gaze_model.fc.in_features, 3)
gaze_model = gaze_model.to(device)

# Checkpoint yükle (state_dict veya 'state_dict' key'ine göre)
ckpt = torch.load(CKPT_PATH, map_location="cpu")
state = ckpt.get("state_dict", ckpt)  # hem plain state_dict hem de dict-wrapper için
missing, unexpected = gaze_model.load_state_dict(state, strict=False)
print("Checkpoint loaded from:", CKPT_PATH)
print("Missing keys:", missing)
print("Unexpected keys:", unexpected)

gaze_model.eval()

# ============================================================
# TEST: sentetik-finetuned modelin REAL test üzerindeki performansı
# ============================================================

test_angles_real = []
with torch.no_grad():
    for xr, tr in tqdm(real_test_loader, desc="REAL Test"):
        xr, tr = xr.to(device), tr.to(device)
        out = gaze_model(xr)
        ang = angular_error_deg(out, tr)
        test_angles_real.append(ang.mean().item())

if len(test_angles_real) > 0:
    mean_ang = float(np.mean(test_angles_real))
    print(f"✅ SYNTHETIC-FINETUNED MODEL on REAL TEST Angular Error: {mean_ang:.2f}°")
else:
    print("⚠️ REAL TEST dataset boş; hiçbir batch işlenmedi.")


Device: cuda
[REAL-test] found 17 videos.


Indexing REAL-test: 100%|██████████| 17/17 [01:01<00:00,  3.59s/it]


REAL-test: 194572 samples
REAL Test samples: 194572
Checkpoint loaded from: /content/drive/MyDrive/dataset_split/best_gaze_synthetic_from_real_lastpt_resnet50.pt
Missing keys: []
Unexpected keys: []


REAL Test: 100%|██████████| 24322/24322 [27:48<00:00, 14.58it/s]

✅ SYNTHETIC-FINETUNED MODEL on REAL TEST Angular Error: 12.33°
